In [1]:
import torch
import sys
sys.path.append('../..')
from transformers import GPTJForCausalLM, AutoTokenizer
import lre.models as models
import lre.functional as functional
import os

device = "cuda:1"
weights = []
biases = []
subjects = []

/home/exia/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Approximate jacobian s-->o for 5->27
animals = ["dog", "duck", "fish", "horse", "mink", "seal", "shark", "trout"]

animal_jacobians = []
for animal in animals:
    K = 10
    animal_jacobian = torch.eye(4096).half().to(device)
    
    kind = "o_o"
    for i in range(26, K, -1):
        weight_path = f"{animal}/{kind}_weight_h_{i}.pt"
        weight = torch.load(weight_path).to(device)
        hs_path = f"{animal}/hs_o_{i}.pt"
        hs = torch.load(hs_path).to(device)[None]
        term = hs.mm(weight)
        animal_jacobian *= term
        
    kind = "s_o"
    for i in range(K, K-1, -1):
        weight_path = f"{animal}/{kind}_weight_h_{i}.pt"
        weight = torch.load(weight_path).to(device)
        hs_path = f"{animal}/hs_h_{i}.pt"
        hs = torch.load(hs_path).to(device)[None]
        term = hs.mm(weight)
        animal_jacobian *= term

    kind = "s_s"
    for i in range(K, 4, -1):
        weight_path = f"{animal}/{kind}_weight_h_{i}.pt"
        weight = torch.load(weight_path).to(device)
        hs_path = f"{animal}/hs_h_{i}.pt"
        hs = torch.load(hs_path).to(device)[None]
        term = hs.mm(weight)
        animal_jacobian *= term

    animal_jacobians.append(animal_jacobian)

animal_jacobians

[tensor([[  -inf, -0.0000, -0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [-0.0000,   -inf, -0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [-0.0000, -0.0000,   -inf,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [-0.0000, -0.0000, -0.0000,  ...,    inf, 0.0000, 0.0000],
         [-0.0000, -0.0000, -0.0000,  ..., 0.0000,    inf, 0.0000],
         [-0.0000, -0.0000, -0.0000,  ..., 0.0000, 0.0000, 0.0303]],
        device='cuda:1', dtype=torch.float16, grad_fn=<MulBackward0>),
 tensor([[     -inf,    0.0000,    0.0000,  ...,    0.0000,   -0.0000,
            -0.0000],
         [  -0.0000,       inf,    0.0000,  ...,    0.0000,   -0.0000,
            -0.0000],
         [  -0.0000,    0.0000, 7652.0000,  ...,    0.0000,   -0.0000,
            -0.0000],
         ...,
         [  -0.0000,    0.0000,    0.0000,  ...,       inf,   -0.0000,
            -0.0000],
         [  -0.0000,    0.0000,    0.0000,  ...,    0.0000,      -inf,
            -0.0000],
         [  -0.0000,    0.0000